# Importing libraries

In [ ]:
import pandas as pd
import pickle
import spacy
import re
import string

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
from glob import glob

from nltk.stem import WordNetLemmatizer


import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import pickle

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Load the Data

In [ ]:
csv_list = glob('reddit*.csv')

In [ ]:
csv_list

# Data Cleaning

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
add_stop_words = ['like', 'im', 'know', 'just', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said', 'your', 'you', 'than',
                 'rskincareaddiction']

stops = stopwords.words('english')
for item in add_stop_words:
    stops.append(item)

In [ ]:
words = stops

def remove_stopwords(text):
    text = str(text)
    text = text.lower()
    split_text = text.split(" ")
    clean_text = [w for w in split_text if not w in words]
    
    return ' '.join(clean_text)
    

In [ ]:
def clean_reviews(text):
    
    text = remove_stopwords(text)
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('\n\n', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('��', '', text)
    text = re.sub('❤️', '', text)
    text = re.sub('☹️', '', text)
    text = re.sub('☺️', '', text)
    text = re.sub('✊', '', text)
    text = re.sub('☀️', '', text)
    text = re.sub(' ͡° ͜ʖ ͡°','', text)
    text = re.sub('♥️♥️', '', text)
    text = re.sub('♥️', '', text)
    text = re.sub('✌', '', text)
    text = re.sub('❤', '', text)
    text = re.sub('✨', '', text)
    text = re.sub('\n\n\n\n', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.translate(str.maketrans('','','’“•–'))
    text = remove_stopwords(text)
    text = wordnet_lemmatizer.lemmatize(text)
    
        
    return text





In [ ]:
def automate(csv_list,):
    all_reviews = []
    for file in csv_list:
        df = pd.read_csv(file)
        df = df.drop(columns=['subreddit','created_utc'])
                df = df.drop(df[df['author'] == 'AutoModerator'].index) 
        df = df.drop(df[df['author'] == 'sunscreenpuppy'].index)
        df = df.drop(df[(df['author'] == 'removed') & (df['author'] == 'deleted')].index)
        df = df.drop(df[df['body'] == 'removed'].index)
        df = df.drop(df[df['body'] == 'deleted'].index)
        
       
        review_data = df.drop(columns=['author'])
        
        review_data['clean_review'] = review_data['body'].apply(lambda x: clean_reviews(x))
        
        cleaned_reviews = list(review_data['clean_review'])
        
        all_reviews.append(cleaned_reviews)
        
    return all_reviews       

In [ ]:
all_reviews = automate(csv_list)

In [ ]:
pd.to_pickle(all_reviews, 'all_reviews.pkl')

# Matching

In [2]:
data = pd.read_pickle('all_reviews.pkl')

In [ ]:
len(data)

In [4]:
data

[['keep skincare products except face wash cosmetic bag beside sofa also keep bag miceller wipes wipe face even dont feel going bathroom wash face watch tv regular skin routine least put moisturizer lot better sticking routine dont go extra effort',
  'absolutely  ive broken horrifically dumb trying random sheet masks ulta definitely little cheap onesthe pricey dr jart brand broke frequently kept trying theyre expensive must better dumb  steer clear sheet mask aisle ',
  'thanks much asian beauty really complicate things  especially names lotion lol appreciate help xd',
  'gp prescribed last 7 years lymecycline three times 12 months 4 months 6 months respectively doxycycline nine months cilest combined pill 7 months differin one year duac three months benzoyl peroxide onoff 4 yearsthese medications prescribed cover many lifestyleskincare changes ive tried last doctor tried refer dermatology rejected tried two different antibiotics couple weeks ago saw doctor see could done see dermatol

In [122]:
skincare_ingredients = pd.read_pickle('skincare_ingrednts.pkl')
product_names = pd.read_pickle('product_names.pkl')

In [123]:
product_names.remove( 'Cleanser ')
skincare_ingredients.remove('acid')
skincare_ingredients.remove('water')

In [124]:
skin_type = ['oily', 'dry', 'combination', 'normal', 'sensitive', 'acne', 
             'hyperpigmentation', 'anti-aging', 'discolouration', 'breakout']

In [125]:
extend_ingredients = ['ha', 'aha', 'bha','vit c', 'snail cream', 
                      'retinoid', 'aza', 
                      'retin-a', 'aloe', 'accutane', 'tret']
extend_products = ['to', 'cetaphil', 'cerave cream', 'cerave pm', 
              'cetaphil non-soap bar','The ordinary’s 10 niacinamide solution', 
              'Paula’s Choice skin perfecting 2 BHA', 'Cetaphil moisturizing lotion',
              'Cetaphil oil control 30 spf moisturizing sunscreen PM', 'cerave foaming facial cleanser',
              'Cetaphil moisturizing cream',
              'the ordinary cold pressed rosehip oil',
              'the ordinary 5 lactic acid solution',
              'inkey list under eye caffeine treatment',
              'Hada Labo premium lotion',
              'TO Niacinamide',
                'to niacinamide + zinc'
              'Hada Labo sunscreen',
              'cosrx snail mucin',
              'Cerave healing ointment',
              'cosrx snail essence',
              'Cerave Resurfacing Retinol',
              'Paula Choice 2 BHA',
              'Cerave Salicylic Acid Renewal Cleanser',
              'La Roche Posay Toleriane Gentle Cleanser',
              'Ole Henriksen Vitamin C eye cream',
              'Ole Henriksen Vitamin C moisturizer',
              'Aveeno Positively Radiant SPF 3',
              'TO glycolic acid toner',
              'Kose Softymo Deep Cleansing Oil',
              'Stridex',
              'Neutrogena hydro boost gel cream ',
                'Neutrogena hydro boost cream',
                   'Neutogena hydro boost'
              'Trader Joes ultra moisturizing hand cream',
              'cerave',
              'Burt bees',
              'PAULAS CHOICE HYALURONIC ACID BOOSTER',
              'First Aid Beauty Coconut Water Cream',
              'The Body Shop Tea Tree Oil',
              'Truly Super Matcha cleanser',
              'Truly Mary Jane Glow serum',
              'Cerave Skin Renewing Vitamin C Serum',
              'Simple Hydrating Gel Cream',
              'Hydrocortisone 1 cream',
              'Simple Micellar Facial Gel Wash',
              'instaNatural Retinol Serum',
              'Purito Centella Green Level Buffet Serum',
              'Stratia Liquid Gold',
              'EltaMD',
              'COSRX Low pH Good Morning Gel Cleanser',
              'Dr. Jart+ Ceramidin Cream',
              'COSRX Honey Ceramide Full Moisture Cream',
              'Dear Klairs Soft Airy UV Essence SPF',
              'Glow Recipe Avocado Eye Mask',
              'Drunk Elephant A-Passioni Retinol Cream',
              'Ole Henriksen Glow2O Dark Spot Toner',
              'TO AHA 30 + BHA 2 Peeling Solution',
              'the ordinary AHA 30 BHA 2 peeling solution',
              'bioderma sensibio gel cleanser',
              'bioderma micellar water',
              'Fresh Soy Face Cleanser',
              'Ole Henrickson Banana Bright Eye Cream',
              'La Roche-Posay Anthelios Sunscreen',
              'cerave Daily Moisturizing Lotion',
              'Dr. Jart Cicapair Tiger Grass Color Correcting Treatment',
              'Laniege Water Sleeping Mask',
              'Olaplex Bond Maintenace Shampoo and Conditioner',
              'Sunday Riley Vitamin C 15 Serum',
              'Summer Fridays Overtime Mask',
              'Biossance Gel Moisturizer',
              'Drunk Elephant jelly cleanser',
              'The Ordinary Niacinamide',
              'Timeless Vitamin C 20',
              'Mario badescu',
              'TO Niacinamide',
              'Cerave Foaming Cleanser',
              'VICHY Mineral 89 serum',
              'No7 Eye Cream',
              'Vaseline',
              'Australian Gold spf50',
              'No7 Lift & Luminate Eye Cream',
              'Retin-A Tret',
              'The Ordinary',
              'Inkey list Q10',
              'TO Azelaic acid',
              'Hada Labo Arbutin lotion',
              'The Ordinary Natural Moisturizing Factors',
              'TO vit C',
              'Osma Bloc - Alum Block 75g',
              'CE ferulic',
              'Drunk Elephant',
              'nars',
              'Stratia Liquid Gold',
              'snail mucin',
              'Biore UV Aqua Rich Watery Essence',
                'Cerave SPF 30',
                'Cerave hydrating cleanser',
                'aquafor',
                'Neutrogena Ultra Gentle Hydrating Cleanser',
                'tatcha water cream',
                'cosrx 96 snail mucin',
                'Thayer Witch Hazel Toner',
                'LaNeige Water Bank Essence',
                'Drunk Elephant C-Firma Day Serum',
                'Boscia Makeup Breakup Cleansing Oil',
                'Dr Jart Ceramidin Lotion',
                   'ole henriksen',
                   'shiseido sunscreen',
                   'Ole Henriksen Truth Serum',
                   'Ole Henriksen Banana Bright',
                   'Olay Regenerist Retinol24',
                   'aztec secret clay mask',
                   'la rocheposay anthelios antishine spf50'
                   'aztec secret clay mask',
                   'pixi glow tonic'
             
]

In [126]:
#extend lists
for item in extend_ingredients:
    skincare_ingredients.append(item)
    
for item in extend_products:
    product_names.append(item)

In [127]:
product_names = set(product_names)

In [128]:
def skin_match(x, match):
    splitreview = x.split(" ")
    setreview = set(splitreview)
    setmatch = set(match)
    
    matches = setreview.intersection(setmatch)
    matches = list(matches)
    
    if len(matches) >= 1:
        match_string = ','.join(matches)
        return match_string
    else:
        return ''

In [129]:
def pi_match(x, match):
    matches = []
    for product in match:
        splitproduct = product.split(" ")
        if len(splitproduct) > 1:
            if product.lower() in x.lower():
                matches.append(product)
        else:
            splitreview = set(x.split(" "))
            splitproduct = set(splitproduct)
            
            m = splitproduct.intersection(splitreview)
            m = list(m)
            if m == []:
                continue
            else:
                matches.append(str(m[0]))
            
    
    if len(matches) >= 1:
        match_string = ','.join(matches)
        return match_string
    else:
        return ''

In [196]:
final_df = pd.DataFrame(columns=['review','skin_type','products','ingredients'])

In [ ]:
for review in data[18]:
    review_items = []
    review_items.append(review)
    review_items.append(skin_match(review, skin_type))
    review_items.append(pi_match(review, product_names))
    review_items.append(pi_match(review, skincare_ingredients))

    final_df.loc[len(final_df)] = review_items


In [189]:
#to check how many items were matched. Just change the value in the dataframe

s = list(final_df['ingredients'])
len([i for i in s if i != ''])

14973

check matches across skin, product, and ingredients. change variable name from t

In [190]:
t = final_df[final_df['ingredients'] != '']

In [191]:
t = t[final_df['skin_type'] != '']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [192]:
t = t[final_df['products'] != '']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [193]:
t

,review,skin_type,products,ingredients
24,thank much appreciate work toy put question\r\...,combination,Hada Labo premium lotion,"alcohol,ceramides,fatty acid,fatty alcohol"
28,well shower pm used cerave hydrating cleanser ...,"dry,oily","cerave,Cerave hydrating cleanser","aloe vera,aloe"
76,read skincare routine would definitely suggest...,oily,Vitamin C Serum,vitamin C
168,one thing worked really well making sure skin ...,"acne,dry,oily",Hyaluronic Acid Serum,"hyaluronic acid,rose oil"
242,look dermatology literature youll find fact co...,acne,Drunk Elephant,"azelaic acid,hyaluronic acid,vitamin C"
...,...,...,...,...
56543,aha probably gylcolic acid lactic acid also c...,"dry,sensitive",Retinol Serum,"lactic acid,retinol,aha,bha,retinoid"
56632,fairly sensitive normaltodry skin palefrecklyr...,"acne,dry,sensitive",cerave,squalane
56664,yep agree anesthesiologists definitely need be...,acne,Cerave healing ointment,"niacinamide,vitamin C"
56722,yes pretty much eliminated body acne tweaks an...,acne,cerave,soap


In [194]:
t = t.reset_index()
t = t.drop(columns=['index'])
t

,review,skin_type,products,ingredients
0,thank much appreciate work toy put question\r\...,combination,Hada Labo premium lotion,"alcohol,ceramides,fatty acid,fatty alcohol"
1,well shower pm used cerave hydrating cleanser ...,"dry,oily","cerave,Cerave hydrating cleanser","aloe vera,aloe"
2,read skincare routine would definitely suggest...,oily,Vitamin C Serum,vitamin C
3,one thing worked really well making sure skin ...,"acne,dry,oily",Hyaluronic Acid Serum,"hyaluronic acid,rose oil"
4,look dermatology literature youll find fact co...,acne,Drunk Elephant,"azelaic acid,hyaluronic acid,vitamin C"
...,...,...,...,...
1140,aha probably gylcolic acid lactic acid also c...,"dry,sensitive",Retinol Serum,"lactic acid,retinol,aha,bha,retinoid"
1141,fairly sensitive normaltodry skin palefrecklyr...,"acne,dry,sensitive",cerave,squalane
1142,yep agree anesthesiologists definitely need be...,acne,Cerave healing ointment,"niacinamide,vitamin C"
1143,yes pretty much eliminated body acne tweaks an...,acne,cerave,soap


In [20]:
t.iloc[3]['products']

'Luna Sleeping Night Oil,Luna Sleeping Night Oil,Luna Sleeping Night Oil,Vitamin C Serum,cetaphil,cerave pm,cerave,Mario badescu,Drunk Elephant,LaNeige Water Bank Essence'

In [195]:
t.to_csv('newmatching_17.csv')

Joining all the reclassified comments into one dataframe

In [200]:
all_matches = glob('newmatching*.csv')

list_ = []
for file in all_matches:
    df = pd.read_csv(file, lineterminator='\n')
    list_.append(df)
final_match = pd.concat(list_, ignore_index=True)

In [201]:
final_match

,Unnamed: 0,review,skin_type,products,ingredients
0,0,moisturize every day things aveeno drugstorebu...,acne,"Vitamin C Serum,The Ordinary","niacinamide,salicylic acid,vitamin C"
1,1,similar routines skin typeconcerns going 28 ye...,"dry,acne",Salicylic Acid Cleanser,"benzoyl peroxide,retinol,salicylic acid,vitami..."
2,2,say anything else primary care appointment ja...,"dry,sensitive",cetaphil,"hyaluronic acid,soap"
3,3,drugstore cleansers general tend harsh irritat...,"acne,oily",cerave,"fragrance,menthol,salicylic acid"
4,4,acne pretty bad growing face back back somewha...,"acne,sensitive","Cerave Foaming Cleanser,cerave",fragrance
...,...,...,...,...,...
23118,1151,dry skin burn lotion applied even without alco...,dry,"cerave,Cerave healing ointment","alcohol,coconut,coconut oil"
23119,1152,thayers hydrating toner nothing acne cerave cl...,acne,cerave,salicylic acid
23120,1153,sunscreenfor much products approach multiple h...,dry,cerave,"amino acids,squalane,urea"
23121,1154,ive used glow tonic impressed formulated prope...,acne,Glow Tonic,"ascorbic acid,fragrance,glycolic acid,salicyli..."


In [202]:
final_match.drop('Unnamed: 0', axis=1, inplace=True)

In [203]:
final_match.to_csv('final_match2.csv')

## Sentiment Analysis

In [228]:
data = pd.read_csv('final_match2.csv', lineterminator='\n')

In [229]:
skincare_sentiment = pd.read_csv('sentiment lexicon.csv', header=None)

In [205]:
skincare_sentiment = skincare_sentiment.drop(skincare_sentiment.columns[2], axis=1)
skincare_sentiment

,0,1
0,severe,-5.83
1,suffered,-5.68
2,developed,-5.57
3,painful,-5.54
4,massive,-5.48
...,...,...
4975,congrats,2.87
4976,beautiful,2.89
4977,congratulations,2.90
4978,gorgeous,2.92


In [206]:
new_dict = dict(zip(skincare_sentiment[0], skincare_sentiment[1]))

In [231]:
analyzer = SentimentIntensityAnalyzer()
analyzer.lexicon.update(new_dict)

def sentiment(text):
    vs = analyzer.polarity_scores(text)
    
    return vs['compound']
    

In [232]:
data['sentiment'] = data['review'].apply(lambda x: sentiment(x))

In [233]:
data

,Unnamed: 0,review,skin_type,products,ingredients,sentiment
0,0,moisturize every day things aveeno drugstorebu...,acne,"Vitamin C Serum,The Ordinary","niacinamide,salicylic acid,vitamin C",0.9798
1,1,similar routines skin typeconcerns going 28 ye...,"dry,acne",Salicylic Acid Cleanser,"benzoyl peroxide,retinol,salicylic acid,vitami...",-0.7380
2,2,say anything else primary care appointment ja...,"dry,sensitive",cetaphil,"hyaluronic acid,soap",-0.9952
3,3,drugstore cleansers general tend harsh irritat...,"acne,oily",cerave,"fragrance,menthol,salicylic acid",0.9569
4,4,acne pretty bad growing face back back somewha...,"acne,sensitive","Cerave Foaming Cleanser,cerave",fragrance,0.6286
...,...,...,...,...,...,...
23118,23118,dry skin burn lotion applied even without alco...,dry,"cerave,Cerave healing ointment","alcohol,coconut,coconut oil",-0.3356
23119,23119,thayers hydrating toner nothing acne cerave cl...,acne,cerave,salicylic acid,0.0155
23120,23120,sunscreenfor much products approach multiple h...,dry,cerave,"amino acids,squalane,urea",0.9870
23121,23121,ive used glow tonic impressed formulated prope...,acne,Glow Tonic,"ascorbic acid,fragrance,glycolic acid,salicyli...",0.9872


In [234]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [212]:
pd.to_pickle(data, 'final_sentiment2.pkl')

# Creating user-item matrix

In [ ]:
user_id = []
product_id = []
sentiment = []
product_name = []
skins = []
reviews = []

for user in range(len(data['review'])):
    products = data.iloc[user,2]
    skin = data.iloc[user,1]
    sent = data.iloc[user,4]
    review = data.iloc[user,0]
    ingredients = data.iloc[user,3]
    split_ing = ingredients.split(',')
    split_products = products.split(',')
    for item in split_products:
        item = item.strip()
        user_id.append(user+1)
        reviews.append(review)
        skins.append(skin)
        sentiment.append(sent)
        if item in product_name:
            item_index = product_name.index(item)
            p_id = product_id[item_index]
            product_name.append(item)
            product_id.append(p_id)
        else:
            product_name.append(item)
            product_id.append(len(product_id)+1)
    for ing in split_ing:
        ing=ing.strip()
        user_id.append(user+1)
        reviews.append(review)
        skins.append(skin)
        sentiment.append(sent)
        if ing in product_name:
            item_index = product_name.index(ing)
            p_id = product_id[item_index]
            product_name.append(ing)
            product_id.append(p_id)
        else:
            product_name.append(ing)
            product_id.append(len(product_id)+1)
    

In [236]:
d = pd.DataFrame({'user_id':user_id, 'product_id':product_id,'product_name':product_name,'skin_type':skins,'rating':sentiment,'review':reviews})

In [237]:
d

,user_id,product_id,product_name,skin_type,rating,review
0,1,1,Vitamin C Serum,acne,0.9798,moisturize every day things aveeno drugstorebu...
1,1,2,The Ordinary,acne,0.9798,moisturize every day things aveeno drugstorebu...
2,1,3,niacinamide,acne,0.9798,moisturize every day things aveeno drugstorebu...
3,1,4,salicylic acid,acne,0.9798,moisturize every day things aveeno drugstorebu...
4,1,5,vitamin C,acne,0.9798,moisturize every day things aveeno drugstorebu...
...,...,...,...,...,...,...
115243,23122,4,salicylic acid,acne,0.9872,ive used glow tonic impressed formulated prope...
115244,23122,5,vitamin C,acne,0.9872,ive used glow tonic impressed formulated prope...
115245,23123,15,cerave,acne,-0.8552,cerave cleanser moisturizer treat acne acne sp...
115246,23123,4,salicylic acid,acne,-0.8552,cerave cleanser moisturizer treat acne acne sp...


In [217]:
pd.to_pickle(d, './skin_dataset3.pkl')

## Aspect based sentimet analysis